In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-19 15:33:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-06-19 15:33:07 (6.62 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [12]:
total_votes_df = vine_df.filter(vine_df["total_votes"]>=20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|     

In [13]:
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"]>=0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [17]:
paid_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] =='Y')
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REKN4PLM5174P|          4|          219|        225|   Y|                N|
| RQOG8IIJX7CKB|          4|           21|         27|   Y|                N|
|R33EJ9PSZR13CM|          4|           26|         30|   Y|                N|
| RD2BFV896F321|          5|           16|         20|   Y|                N|
|R37GSXLA11XF8N|          5|           30|         34|   Y|                N|
| RQBQYBM9VMSH4|          5|           19|         20|   Y|                N|
|R2YE4F9ESKFWEK|          3|           20|         23|   Y|                N|
|R1E4L7HHB8TBP9|          4|           80|         92|   Y|                N|
|R2C2WXDXMZL005|          2|           20|         25|   Y|                N|
|R1PPY4U0K03KTE|          5|           31|         38|   Y|     

In [18]:
unpaid_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] =='N')
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [20]:
paid_reviews_df = paid_reviews_df("review_id").count()

TypeError: ignored